# Agentic RAG

Агент, который может использовать RAG для ответа на вопросы.
Ключевым отличием от классического RAG является опциональность поиска. Модель сама решает - отвечать ли на вопрос пользователя с использованием внешнего источника знаний или самостоятельно.

## Retriever
### (Опционально) подключение GigaLogger (beta)

GigaLogger - утилита для логирования запросов и отладки LLM на базе LangFuse.

In [22]:
import getpass
import os

def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")

# If you want to use gigalogger
_set_env("GIGALOGGER_PUBLIC_KEY")
_set_env("GIGALOGGER_SECRET_KEY")

### Загрузка данных
Выгрузим несколько постов про GigaChat с Хабра и добавим их в векторную базу данных

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.vectorstores.chroma import Chroma

# Используйте токен, полученный в личном кабинете из поля Авторизационные данные

def get_giga():
    return GigaChat(
            model="GigaChat-Pro"
            credentials="credentials ..."
            )    


urls = [
    "https://habr.com/ru/companies/sberdevices/articles/794773/",
    "https://habr.com/ru/companies/sberdevices/articles/792660/",
    "https://habr.com/ru/companies/sberdevices/articles/777578/",
    "https://habr.com/ru/companies/sberbank/articles/773180/",
    "https://habr.com/ru/companies/sberdevices/articles/790470/"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)


In [32]:
giga_embeddings = GigaChatEmbeddings(credentials="...")

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=giga_embeddings
)

retriever = vectorstore.as_retriever()

Теперь создадим тул, который дает модели возможность воспользоваться ретривером

In [11]:
from langchain.tools.retriever import create_retriever_tool

from langgraph.prebuilt import ToolExecutor

tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Выполняет поиск по статьям про GigaChat на Хабре",
)

tools = [tool]

tool_executor = ToolExecutor(tools)

Проверим, что векторная БД возвращает релевантные данные.

In [12]:
tool("Что такое SignFlow?")

/Users/19563044/Documents/Giga/gigagraph/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'как отсутствие жеста). Подробнее о создании набора данных и его характеристиках мы ранее писали в статье «\u200e\u200eSlovo и русский жестовый язык».Семейство моделей SignFlowНа данный момент в открытом доступе семейство моделей SignFlow содержит 2 модели — SignFlow-R для распознавания РЖЯ и SignFlow-A для распознавания американского жестового языка. В основе каждой из них лежит видео-трансформер mVITv2-S.\n\nпросмотров2.8KБлог компании SberDevicesОбработка изображений*Accessibility*Машинное обучение*Искусственный интеллектВсем привет! Меня зовут Капитанов Александр, я отвечаю за направление компьютерного зрения в SberDevices. В этой статье я расскажу о том, как моя команда Vision RnD разработала серию моделей SignFlow, обеспечивающих перевод жестового языка на русский и американский английский в\n\nи косинусный с 20-й и до конца обучения. Графики loss-функции и метрики Mean Accuracy для процесса обучения лучшей модели представлены ниже.Loss-функция в процессе обученияМетрика Mean-Acc

## Agent state
 
Мы определим граф

Объект `state` будет передаваться каждому узлу (ноде) графа.

Нашим стейтом будет список `messages`.

Каждый узел нашего графа будет туда добавть сообщение.

In [13]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Nodes and Edges

Опишем граф нашего агента с RAG следующим образом:

* `state` это набор сообщений  (`messages`)
* Каждый узел графа обновляет `state` (добавляет сообщение)
* Рёбра с условмиями (`conditional edges`) определяют, к какому узлу графа будет выполнен переход


In [14]:
import operator
from typing import Annotated, Sequence, Type, TypedDict

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import GigaChat
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.utils.function_calling import convert_to_gigachat_function

from langgraph.prebuilt import ToolInvocation

### Edges


def should_retrieve(state: Type[AgentState]) -> str:
    """
    Decides whether the agent should retrieve more information or end the process.

    This function checks the last message in the state for a function call. If a function call is
    present, the process continues to retrieve information. Otherwise, it ends the process.

    Args:
        state (messages): The current state

    Returns:
        str: A decision to either "continue" the retrieval process or "end" it
    """
    
    print("---DECIDE TO RETRIEVE---")
    messages = state["messages"]
    last_message = messages[-1]


    if not isinstance(last_message, BaseMessage):
        raise Exception("Last message is not a BaseMessage instance")

    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        print("---DECISION: DO NOT RETRIEVE / DONE---")
        return "end"
    # Otherwise there is a function call, so we continue
    else:
        print("---DECISION: RETRIEVE---")
        return "continue"

In [15]:
def grade_documents(state: Type[AgentState]):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (messages): The current state

    Returns:
        str: A decision for whether the documents are relevant or not
    """


    llm = get_giga()

    # Prompt
    prompt = PromptTemplate(
        template="""Вы — оценщик, оценивающий соответствие полученного документа вопросу пользователя. \п
         Вот полученный документ: \n\n {context} \n\n
         Вот вопрос пользователя: {question} \n
         Если документ содержит ключевые слова или семантическое значение, связанное с вопросом пользователя, оцените его как релевантный. \п
         Дайте двоичную оценку "yes" или "no", чтобы указать, имеет ли документ отношение к вопросу.""",
        input_variables=["context", "question"],
    )

    # Chain
    #chain = prompt | llm_with_tool | parser_tool
    chain = prompt | llm

    
    messages = state["messages"]
    last_message = messages[-1]

    print(f"---CHECKING RELEVANCE FOR DOCS--- last_message={last_message}")

    question = messages[0].content

    docs = last_message.content
    
    score = chain.invoke(
        {"question": question, 
         "context": docs}
    )
    print(f"---SCORE--- {score}")
    

    grade = score.content

    if grade == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return "yes"
    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        print(grade)
        return "no"

In [16]:
### Nodes


def agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response apended to messages
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    
    print("Agent received:", messages)
    model = get_giga()
    
    functions = [convert_to_gigachat_function(t) for t in tools]
    model = model.bind_functions(functions)

    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    
    """
    ----- # due to current code at langchain_community/chat_models/gigachat.py def _convert_dict_to_message(message: Messages) -> BaseMessage:
        if not isinstance(response, BaseMessage):
        raise Exception("Last message is not a BaseMessage instance")
    """


    # If there is no function call, then we finish
    if "function_call" in response.additional_kwargs: 
        if function_call := response.additional_kwargs["function_call"]:
            from gigachat.models.function_call import FunctionCall
            if isinstance(function_call, FunctionCall): 
                response.additional_kwargs["function_call"] = dict(function_call) 
    """
    ----- # due to current code at langchain_community/chat_models/gigachat.py def _convert_dict_to_message(message: Messages) -> BaseMessage:
    """

    return {"messages": [response]}

def retrieve(state):
    """
    Uses tool to execute retrieval.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with retrieved docs
    """
    print("---EXECUTE RETRIEVAL---")
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call

    if function_call := last_message.additional_kwargs["function_call"]:
        from gigachat.models.function_call import FunctionCall
        if isinstance(function_call, FunctionCall): 
            # due to current code at langchain_community/chat_models/gigachat.py def _convert_dict_to_message(message: Messages) -> BaseMessage:
            last_message.additional_kwargs["function_call"] = dict(function_call) 

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=last_message.additional_kwargs["function_call"]["arguments"]
    )

    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

def rewrite(state):
    """
    Transform the query to produce a better question.
    
    Args:
        state (messages): The current state
    
    Returns:
        dict: The updated state with re-phrased question
    """
    
    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    
    question = messages[0].content

    print(f"---TRANSFORM QUERY---messages={messages}\n-----\n question={question}")

    msg = [HumanMessage(
        content=f""" \n 
        Пользователь ищет информацию в статьях. Пользователь формулирует поисковые вопросы к базе знаний.\n 
        Здесь изначальный вопрос пользователя:
        \n ------- \n
        {question} 
        \n ------- \n
        Перефразируй изначальный вопрос пользователя так, чтобы вопрос был полным, завершенным и точным""",
        )]

    # Grader
    model = get_giga()
    response = model.invoke(msg)
     
    print("---TRANSFROM QUERY RESPONSE--- response =", response)

    return {"messages": [response]}

def generate(state):
    """
    Generate answer

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    print("---GENERATE---")
    messages = state["messages"]


    question = messages[0].content

    # LLM
    llm = get_giga()

    # Prompt
    prompt = PromptTemplate(
        template=""", Ты ассистент, который отвечает на вопрос пользователя. 
            Ответь на вопрос пользователя исходя из контекста.
            Если ты не знаешь ответа, просто скажи, что не знаешь, не пытайся придумать ответ. \п
            Вот полученный контекст: \n\n {context} \n\n
            Вот вопрос пользователя: {question} \n
            для ответа на вопрос
            """,
        input_variables=["context", "question"],
    )

    # Chain
    chain = prompt | llm

    response = chain.invoke(
        {"question": question, 
            "context": messages}
)
    return {"messages": [response]}

## Graph

* Запуск агента, `call_model`
* Агент принимает решение вызвать ли функцию
* Если да, то переходим к `action` чтобы вызвать функцию (ретривер)
* Затем вызываем агента с результатом работы функции, добавленным в сообщения (`state`)

In [17]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # retrieval
workflow.add_node("generate", generate)  # retrieval

In [18]:
# Call agent node to decide to retrieve or not
workflow.set_entry_point("agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    should_retrieve,
    {
        # Call tool node
        "continue": "retrieve",
        "end": END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
    {
        "yes": "generate",
        "no": "rewrite",  
    },
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
app = workflow.compile()

In [19]:
import pprint

from langchain_core.messages import HumanMessage


def qna(question):
    inputs = {
        "messages": [
            HumanMessage(
                content=question
            )
        ]
    }
    for output in app.stream(inputs):
        for key, value in output.items():
            pprint.pprint(f"Output from node '{key}':")
            pprint.pprint("---")
            pprint.pprint(value, indent=2, width=80, depth=None)
        pprint.pprint("\n---\n")

## QnA, когда нужен дополнительный поиск

In [20]:
qna("Какие определения агентов приводятся в статьях на Хабре?")

---CALL AGENT---
Agent received: [HumanMessage(content='Какие определения агентов приводятся в статьях на Хабре?')]


Giga generation stopped with reason: function_call


---DECIDE TO RETRIEVE---
---DECISION: RETRIEVE---
"Output from node 'agent':"
'---'
{ 'messages': [ AIMessage(content='', additional_kwargs={'function_call': {'name': 'retrieve_blog_posts', 'arguments': {'query': 'агент'}}, 'functions_state_id': 'e0fd49a1-df5f-4bf3-8907-7f5e96d9a295'}, response_metadata={'token_usage': Usage(prompt_tokens=78, completion_tokens=28, total_tokens=106), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'function_call'}, id='run-8f99133f-8bd2-4806-987f-b219a3ce04c3-0', tool_calls=[{'name': 'retrieve_blog_posts', 'args': {'query': 'агент'}, 'id': 'c69a60b3-9c06-41cd-9060-9824dc68d4ff'}])]}
'\n---\n'
---EXECUTE RETRIEVAL---
---CHECKING RELEVANCE FOR DOCS--- last_message=content="Билл Гейтс говорит, что агенты — это тип программного обеспечения, который реагирует на естественный язык и может выполнять множество различных задач на основе знаний пользователя.\xa0Я бы дополнил определение агентов следующим тезисом:«Агент – это программа, которая способна вз

## QnA, когда дополнительный поиск не требуется

In [21]:
qna("Что такое ИИ-агент?")

---CALL AGENT---
Agent received: [HumanMessage(content='Что такое ИИ-агент?')]
---DECIDE TO RETRIEVE---
---DECISION: DO NOT RETRIEVE / DONE---
"Output from node 'agent':"
'---'
{ 'messages': [ AIMessage(content='Искусственный интеллект (ИИ) — это область компьютерной науки, которая занимается созданием интеллектуальных машин и программ, способных выполнять задачи, обычно связанные с человеческим интеллектом. ИИ-агент — это программа или система, которая использует методы искусственного интеллекта для выполнения определенных задач.\n\nИИ-агенты могут быть использованы в различных областях, таких как медицина, финансы, образование, транспорт и многие другие. Они могут выполнять различные функции, включая распознавание образов, обработку естественного языка, принятие решений и многое другое.\n\nОдним из примеров ИИ-агента является Siri от Apple. Это виртуальный помощник, который может отвечать на вопросы, отправлять сообщения, устанавливать напоминания и выполнять другие задачи. Другим пр